
### Artifical Neural Networks - Churn Modelling

- Fully Connected Neural Network: Layers are fully connected layers
- An input vector containing  different features and predict an outcome which is a binary variable i.e. for ***classification***

##### Understanding the [Dataset](../dataFiles/Churn_Modelling.csv)

- Dataset of a bank with information about customers
- Last column i.e. Exited is the dependent variable tells yes/no - the customer stayed or exited the bank
- Bank observed customers for a period of time and gathered outcomes in the dependent variable to get the correlation between the independent features and the dependent variable
- They intent to deploy the model on new  customers to encourage customers to stay with  the bank
- The given dataset will be used to train the model and then use the model on future customers as well as predicting the probability that a given customer leaves the bank.

In [40]:
# Install tensorflow
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


#### Importing the Libraries

In [41]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [42]:
tf.__version__

'2.13.0'

### Part 1 - Data preprocessing

#### Importing the dataset
- exclude column which will have no impact on the outcome i.e. stay or leave (RowNumber, Customerid, Surname)

In [43]:
dataset = pd.read_csv('../dataFiles/Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [44]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [45]:
print(y)

[1 0 1 ... 1 1 0]


#### Encoding categorical data

- encode gender column using label encoding i.e. o and 1 for Gender has a relationship order
- Encode 'Geography' column using OneHotEncoding as data has no relationship order between categories
- No missing data so no need to take care of that

 ***Label encoder of the Gender column***

In [46]:


#label encoding for gender column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2]) # [:, 2] all rows and column 2

***OneHotEncoding for the Geography column***

In [47]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


***OneHotEncoding the 'Georgraphy' Column***

In [74]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [49]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


***Splitting the dataset into  the Training set and Test Set***


In [50]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=0)

##### Feature Scaling - it is compulsory for deep learning

In [51]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

### Part 2 - Building the ANN

- Initialise the ANN as a sequence of layers
- Add the input layer and the first hidden layer composed a certian number of neurons
- add a second hidden layer to build a deep learning model instead of a shallow learning model
- finally add the output layer which will predict our output


***Initialising the ANN***

In [52]:
# create a variable for an ANN network using the sequential class for a sequence of layers
ann = tf.keras.models.Sequential()


***Adding the input layer and  the first hidden layer***

In [53]:
# add as an object of the dense class
#unit = number of neurons - number depends on experimentation with different hyper parameters e.g. 6
#activation function - must be the rectifier function with code name 'relu'
ann.add(tf.keras.layers.Dense(units =6, activation='relu'))

***Adding the second hidden layer***
- similar to the above

In [54]:
ann.add(tf.keras.layers.Dense(units =6, activation='relu'))

***Adding the output layer***
- units depend on the output dimension e.g. in this  example we only need a binary value hence 1 neuron
- the value of the activation function it needs to be a sigmoid function as it allows the predictions and
- probability and the binary output is 1, this will also give us the probability that the customer leaves the bank
- for non binary classification the activation for the output layer should be **softmax** instead of ***sigmoid***

In [55]:

ann.add(tf.keras.layers.Dense(units =1, activation='sigmoid'))

### Part 3 - Training the ANN

***Compiling the ANN***
- Compile the ANN with an optimiser, loss function and a metric for accuracy because we are performing classification
- Train the ANN on the training set over a certain number of Epochs

- stochaistic gradient descent allows us to update the weights in order to reduce the error (loss) between the predictions and the real result
- Best optimiser is one which can perform stochaistic gradient descent - Adam optimiser is the best
- Loss function is computes the difference between the predictions and the real result - loss function must always be **'binary_crossentropy'** if we are dealing with a binary outclassification. For other ***classifications** it needs to be **'categorical_crossentropy'**
- Accuracy is the final evaluation metric - the main metric is accuracy

In [56]:
#parameters: optimizer, loss function and evaluation metric

ann.compile(optimizer ='adam', loss ='binary_crossentropy' , metrics =['accuracy'])

***Training the ANN on the Training Dataset***
- use batch learning as it is a lot more efficient when training a model, classic batch size = 32
- epochs improve the performance of the mode over time

In [57]:
ann.fit(x_train, y_train, batch_size =32, epochs =100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6360 - accuracy: 0.6659
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4993 - accuracy: 0.7965
Epoch 3/100
184/250 [=====================>........] - ETA: 0s - loss: 0.4600 - accuracy: 0.8008

250/250 [==============================] - 0s 1ms/step - loss: 0.4598 - accuracy: 0.7972
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4437 - accuracy: 0.8011
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4325 - accuracy: 0.8066
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4212 - accuracy: 0.8112
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4044 - accuracy: 0.8259
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3868 - accuracy: 0.8359
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3733 - accuracy: 0.8456
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3646 - accuracy: 0.8489
Epoch 11/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3597 - accuracy: 0.8510
Epoch 12/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3560 - accuracy: 0.8535

### Part 4: Making the predictions and evaluating the model

#### Challenge
Use the above ANN to predict if a customer with the following information will leave the back:
- Georgraphy: France
- CreditScore:600
- Gender: Male
- Age: 40 Years Old
- Tenure: 3 Years
- Balance:$60000
- Number of products: 2
- Does this customer have a credit card? Yes
- Is this customer an Active Member: Yes
- Estimated Salary: $50000
- Should be way goodbye to this customer? 

#### Predicting the result of a single observation

In [82]:
test_sample = [[600,'France', 'Male', 40, 3, 60000, 2,1,1,50000]] 
#input must be a 2d array whether it is a single outcome or multiple
# prediction is returned in form of a probability
# apply the scaling which was used to train the mode
print(ann.predict(sc.transform([[1, 0,0, 600, 1, 40, 3, 60000, 2,1,1,50000]]))>0.5)

1/1 [==============================] - 0s 18ms/step
[[False]]


***Outcome is that the customer will not leave the Bank***

***Predicting Test set results***

In [85]:
y_pred = ann.predict(x_test)
#convert the predicted to binary outcomes
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test),1)),1))

 1/63 [..............................] - ETA: 1s

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


***Making the confusion Matrix***

In [86]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1589    6]
 [ 350   55]]


0.822